In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sqlalchemy import text
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from database.connection import get_db_manager

In [ ]:
# CONFIGURACIÓN PARA RADAR DE EQUIPOS
TABLE_TYPE = 'domestic'
LEAGUES = ['ESP-La Liga']
SEASON = '2526'
TARGET_TEAM = 'Atlético Madrid'

# Configuración de colores del Atlético Madrid
ATM_COLORS = ['#FF6B6B', '#FFFFFF']  # Rojo y blanco

SAVE_PATH_BASE = "../images/mfichajes/"
os.makedirs(SAVE_PATH_BASE, exist_ok=True)

In [ ]:
# Consulta para obtener datos de equipos
db = get_db_manager()

league_list_str = "', '".join(LEAGUES)
query = f"""
SELECT 
    unique_team_id,
    team_name,
    league,
    season,
    fbref_metrics,
    understat_metrics
FROM footballdecoded.teams_{TABLE_TYPE} t
WHERE t.league IN ('{league_list_str}')
AND t.season = '{SEASON}'
ORDER BY t.team_name
"""

df_teams_raw = pd.read_sql(query, db.engine)
db.close()

print(f"Equipos extraídos: {df_teams_raw.shape[0]} registros, {df_teams_raw.shape[1]} columnas")
df_teams_raw.head()

In [ ]:
def extract_team_metrics(df, col_name):
    """Extrae métricas numéricas de columnas JSON para equipos"""
    result = pd.DataFrame(index=df.index)
    
    all_keys = set()
    for _, row in df.iterrows():
        if isinstance(row[col_name], dict):
            all_keys.update(row[col_name].keys())
    
    for key in all_keys:
        values = []
        for _, row in df.iterrows():
            if isinstance(row[col_name], dict) and key in row[col_name]:
                raw_value = row[col_name][key]
                converted_value = _convert_to_float(raw_value)
                values.append(converted_value)
            else:
                values.append(np.nan)
        
        # Para equipos, requerimos al menos 3 valores válidos
        valid_count = pd.Series(values).notna().sum()
        if valid_count >= 3:
            result[key] = values
    
    return result

def _convert_to_float(value):
    """Convierte valores a float de manera segura"""
    if isinstance(value, (int, float)):
        return float(value)
    
    if value is None or pd.isna(value):
        return np.nan
    
    if isinstance(value, str):
        if value.strip() == '':
            return np.nan
        
        if value.lower().strip() in ['nan', 'none', 'null', '-']:
            return np.nan
        
        try:
            return float(value)
        except (ValueError, TypeError):
            return np.nan
    
    return np.nan

# Extraer métricas de equipos
team_fbref_metrics = extract_team_metrics(df_teams_raw, 'fbref_metrics')
team_understat_metrics = extract_team_metrics(df_teams_raw, 'understat_metrics')

print(f"FBref equipos: {team_fbref_metrics.shape[1]} métricas")
print(f"Understat equipos: {team_understat_metrics.shape[1]} métricas")

In [ ]:
# Combinar todas las métricas de equipos
all_team_metrics = pd.concat([team_fbref_metrics, team_understat_metrics], axis=1)

# Calcular percentiles para equipos (comparación entre equipos de La Liga)
team_percentiles = all_team_metrics.rank(pct=True) * 98 + 1
team_percentiles = team_percentiles.round(0).astype('Int64')
team_percentiles.columns = [f'{col}_pct' for col in team_percentiles.columns]

# DataFrame final de equipos
base_team_cols = ['unique_team_id', 'team_name', 'league', 'season']

df_teams_final = pd.concat([
    df_teams_raw[base_team_cols],
    all_team_metrics,
    team_percentiles
], axis=1)

print(f"DataFrame final de equipos: {df_teams_final.shape[0]} filas, {df_teams_final.shape[1]} columnas")
print(f"\nEquipos disponibles:")
for team in sorted(df_teams_final['team_name'].unique()):
    print(f"  - {team}")

In [ ]:
# Explorar métricas disponibles para equipos
print("MÉTRICAS DISPONIBLES PARA EQUIPOS:")
print("=" * 50)

team_metrics_sorted = sorted(list(all_team_metrics.columns))
for i, metric in enumerate(team_metrics_sorted, 1):
    print(f"{i:2d}. {metric}")

print(f"\nTotal métricas: {len(team_metrics_sorted)}")

In [ ]:
# SELECCIÓN DE MÉTRICAS PARA EL RADAR DEL EQUIPO
# PUEDES MODIFICAR ESTAS MÉTRICAS SEGÚN TUS PREFERENCIAS

# Métricas sugeridas (10 métricas principales)
team_radar_metrics = [
    'goals',                    # Goles anotados
    'expected_goals',           # xG - Goles esperados
    'goals_against',            # Goles en contra (se invertirá)
    'shots_on_target',          # Tiros a puerta
    'pass_completion_pct',      # % Pases completados
    'progressive_passes',       # Pases progresivos
    'tackles_won',              # Entradas ganadas
    'interceptions',           # Intercepciones
    'clearances',              # Despejes
    'yellow_cards'             # Tarjetas amarillas (se invertirá)
]

team_radar_titles = [
    'Goals',                   # Goles
    'Expected Goals',          # xG
    'Defensive Solidity',      # Solidez defensiva (menos goles = mejor)
    'Shots on Target',         # Tiros a puerta
    'Pass Accuracy %',         # Precisión de pases
    'Progressive Passes',      # Pases progresivos
    'Tackles Won',             # Entradas ganadas
    'Interceptions',          # Intercepciones
    'Clearances',             # Despejes
    'Discipline'              # Disciplina (menos tarjetas = mejor)
]

# Métricas que deben invertirse (menor es mejor)
invert_metrics = ['goals_against', 'yellow_cards']

# Verificar que todas las métricas están disponibles
available_metrics = [m for m in team_radar_metrics if m in all_team_metrics.columns]
missing_metrics = [m for m in team_radar_metrics if m not in all_team_metrics.columns]

print(f"Métricas disponibles: {len(available_metrics)}/{len(team_radar_metrics)}")
if missing_metrics:
    print(f"Métricas no disponibles: {missing_metrics}")

print(f"\nMétricas seleccionadas para el radar:")
for i, (metric, title) in enumerate(zip(team_radar_metrics, team_radar_titles), 1):
    status = "✓" if metric in available_metrics else "✗"
    invert = " (invertida)" if metric in invert_metrics else ""
    print(f"{i:2d}. {status} {title}{invert}")

In [ ]:
# Encontrar el Atlético Madrid
atletico_data = df_teams_final[df_teams_final['team_name'] == TARGET_TEAM]

if len(atletico_data) > 0:
    atletico_id = atletico_data.iloc[0]['unique_team_id']
    print(f"✓ {TARGET_TEAM} encontrado")
    print(f"  ID: {atletico_id}")
    
    # Mostrar los valores de las métricas seleccionadas
    print(f"\nMétricas del {TARGET_TEAM}:")
    print("=" * 40)
    
    for metric, title in zip(team_radar_metrics, team_radar_titles):
        if metric in available_metrics:
            raw_value = atletico_data.iloc[0][metric]
            percentile = atletico_data.iloc[0][f'{metric}_pct']
            print(f"{title:20s}: {raw_value:8.2f} (percentil {percentile})")
        else:
            print(f"{title:20s}: No disponible")
            
else:
    print(f"✗ {TARGET_TEAM} no encontrado")
    print("Equipos disponibles:")
    for team in sorted(df_teams_final['team_name'].unique()):
        print(f"  - {team}")

In [ ]:
# Generar el radar del equipo
if len(atletico_data) > 0:
    from viz.swarm_radar import create_player_radar
    from viz.stats_radar import create_stats_table, combine_radar_and_table
    from IPython.display import Image as IPImage, display
    
    # Usar métricas disponibles
    final_metrics = [m for m in team_radar_metrics if m in available_metrics]
    final_titles = [t for m, t in zip(team_radar_metrics, team_radar_titles) if m in available_metrics]
    
    print(f"Generando radar del {TARGET_TEAM} con {len(final_metrics)} métricas...")
    
    try:
        # Crear radar del equipo
        create_player_radar(
            df_data=df_teams_final,
            player_1_id=atletico_id,
            metrics=final_metrics,
            metric_titles=final_titles,
            team_colors=ATM_COLORS,
            save_path=f"{SAVE_PATH_BASE}Atletico_Madrid_team_radar.png",
            use_swarm=False,
            show_plot=False
        )
        
        # Crear tabla de estadísticas
        create_stats_table(
            df_data=df_teams_final,
            player_1_id=atletico_id,
            metrics=final_metrics,
            metric_titles=final_titles,
            team_colors=ATM_COLORS,
            save_path=f"{SAVE_PATH_BASE}Atletico_Madrid_team_table.png",
            team_logos={TARGET_TEAM: '../logos/LaLiga/Atletico_Madrid.png'},
            footer_text=f"Team Stats & Percentiles\nvs ESP-La Liga Teams 2025/26",
            show_plot=False
        )
        
        # Combinar radar y tabla
        combine_radar_and_table(
            radar_path=f"{SAVE_PATH_BASE}Atletico_Madrid_team_radar.png",
            table_path=f"{SAVE_PATH_BASE}Atletico_Madrid_team_table.png",
            output_path=f"{SAVE_PATH_BASE}Atletico_Madrid_team_combined.png"
        )
        
        print(f"✓ Radar del equipo completado: Atletico_Madrid_team_combined.png")
        display(IPImage(f"{SAVE_PATH_BASE}Atletico_Madrid_team_combined.png"))
        
    except Exception as e:
        print(f"✗ Error generando radar del equipo: {str(e)}")
        
else:
    print("No se puede generar el radar sin datos del Atlético Madrid")